You must run this notebook locally after creating an LMStudio Server (download it here):
- https://lmstudio.ai/

In [ ]:
%pip install langchain langchain_openai --upgrade

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import Field, BaseModel
from typing import List

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

In [ ]:
# Article Outline:
class ArticleSectionOutline(BaseModel):
    title: str = Field(description="The title of the section")

class ArticleOutline(BaseModel):
    title: str = Field(description="The title of the article")
    sections: List[ArticleSectionOutline] = Field(description="All of the sections of the article")

parser = PydanticOutputParser(pydantic_object=ArticleOutline)
format_instructions = parser.get_format_instructions()

llm: ChatOpenAI = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    temperature=0,
    api_key="not-needed",
    model_kwargs={"stop": ['```']},
)

article_plan = llm.invoke(
    [
        SystemMessage(
            content=f"""Act as a SEO specialist. 
                  You are very specialised in generating highly optimized SEO briefs for content marketers. 
                  You must respond in JSON format only
                  {format_instructions}

                  You must finish it with }} ```
                  """
        ),
        HumanMessage(
            content="""I want you to generate an effective article plan for me on Digital Marketing
            \n```"""
        ),
    ]
)

In [ ]:
article_outline = parser.parse(article_plan.content)

In [ ]:
article_outline.sections

## Additional Resources For JSON Parsing:

- https://github.com/noamgat/lm-format-enforcer
- https://github.com/ollama/ollama/tree/main/examples/python-json-datagenerator